In [ ]:
###############################
# 1. Import Required Libraries
###############################
import os
import numpy as np
import pandas as pd
import torch

# Transformers, NLP
from transformers import (BertTokenizer, BertForSequenceClassification, 
                          Trainer, TrainingArguments, BertModel)

# Sklearn & Model-Related
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, 
                             recall_score, f1_score, classification_report)
from xgboost import XGBClassifier

In [ ]:
###############################
# 2. Load & Prepare Data
###############################
# Assumes we have:
#   1) structured_df: Structured data with numeric/categorical features + target
#   2) text_df: DataFrame with an ID column, textual content (e.g., 'TEXT_COL'),
#               and possibly the same target column if relevant

print("=== Loading Structured Data ===")
structured_path = "final_cleaned_data_phase4.csv"  # Example from Phase 4 output
structured_df = pd.read_csv(structured_path)

print("Structured Data Shape:", structured_df.shape)
print(structured_df.head(2))

print("\n=== Loading Textual Data ===")
text_path = "legal_text_data.csv"  # A CSV containing ID and TEXT_COL
text_df = pd.read_csv(text_path)
print("Text Data Shape:", text_df.shape)
print(text_df.head(2))

# Merge text + structured if needed (assuming a common 'CASE_ID' or similar)
# If they are already combined, skip merge.
merged_df = pd.merge(structured_df, text_df, on='CASE_ID', how='inner')
print("\nMerged Data Shape:", merged_df.shape)

In [ ]:
###############################
# 3. Fine-Tuning BERT on Text
###############################
# Example classification: Predict 'BIAS_LABEL' in the text 
# (where 1 = Biased text, 0 = Non-biased text).
# If your target is something else, adapt accordingly.

print("\n=== Sub-Phase 5.1: Fine-Tune BERT for Text Classification ===")

# Step 3.1: Define the Hugging Face tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# For demonstration, assume we have a 'BIAS_LABEL' column in merged_df
# holding 0/1 for each piece of text, or "Biased"/"Not Biased" to encode.
merged_df['BIAS_LABEL'] = merged_df['BIAS_LABEL'].astype(int)  # ensure numeric

# Step 3.2: Train / Val Split for text
train_text, val_text, y_train_text, y_val_text = train_test_split(
    merged_df['TEXT_COL'].values, 
    merged_df['BIAS_LABEL'].values, 
    test_size=0.2,
    random_state=42,
    stratify=merged_df['BIAS_LABEL']
)

# Step 3.3: Tokenize
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors='pt'
    )

train_encodings = tokenize_function(train_text.tolist())
val_encodings = tokenize_function(val_text.tolist())

# Step 3.4: Create Torch Dataset
class BiasTextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = BiasTextDataset(train_encodings, y_train_text)
val_dataset   = BiasTextDataset(val_encodings,   y_val_text)

# Step 3.5: Define Model & Trainer
from transformers import Trainer, TrainingArguments, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./bert_finetune_output',
    evaluation_strategy='epoch',
    num_train_epochs=2,  # or more
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./bert_logs',
    logging_steps=50,
    save_steps=200,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Step 3.6: Fine-Tune
print("\n=== Fine-Tuning BERT ===")
trainer.train()
trainer.save_model("./bert_finetuned")
print("Fine-tuned BERT saved at './bert_finetuned'")

In [ ]:
###############################
# 4. Extract BERT Embeddings
###############################
# We'll use the fine-tuned BERT's base embeddings for each piece of text,
# then combine with structured features for advanced modeling.

print("\n=== Extract BERT Embeddings for Full Dataset ===")
# Reload fine-tuned model for embedding extraction
from transformers import BertModel

base_bert = BertModel.from_pretrained("./bert_finetuned")
base_bert.eval()

# We'll define a helper to get CLS embeddings for each text row
def get_cls_embedding(text):
    # Tokenize single input
    encoding = tokenizer(
        text, 
        padding=True, 
        truncation=True, 
        max_length=256, 
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = base_bert(**encoding)
        cls_embed = outputs.last_hidden_state[:, 0, :]  # [CLS] token
    return cls_embed.squeeze().numpy()  # shape (768,)

# For demonstration, embed entire dataset
all_texts = merged_df['TEXT_COL'].values
embed_array = []
for txt in all_texts:
    cls_vec = get_cls_embedding(txt)
    embed_array.append(cls_vec)
    
embed_array = np.stack(embed_array, axis=0)  # shape (N, 768)
print("BERT Embeddings Shape:", embed_array.shape)

# Step 4.1: Combine with Structured Data
# Example structured columns: [AGE, CRIMHIST, NEWRACE_Black, ...], excluding the textual columns
structured_cols = [col for col in merged_df.columns 
                   if col not in ['TEXT_COL', 'BIAS_LABEL'] and 'CASE_ID' not in col]
structured_features = merged_df[structured_cols].values  # shape (N, M)

# Final combined feature vector
X_combined = np.concatenate([embed_array, structured_features], axis=1)
print("Combined Feature Shape:", X_combined.shape)

# Our final target (if e.g., DISPOSIT_ENCODED from Phase 4) or any other label
y_combined = merged_df['DISPOSIT_ENCODED'].values  # or BIAS_LABEL, or another target


In [ ]:
###############################
# 5. Advanced Modeling + Fairness
###############################
# Sub-Phase 5.3 & 5.4

print("\n=== Training Advanced Model (e.g., XGBoost) on Combined Features ===")

# Step 5.1: Split data
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(
    X_combined, y_combined, 
    test_size=0.2, 
    random_state=42,
    stratify=y_combined
)

# Step 5.2: Train XGBoost with potential hyperparameter tuning
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_comb, y_train_comb)

# Evaluate
y_pred_comb_test = xgb_model.predict(X_test_comb)
acc_test = accuracy_score(y_test_comb, y_pred_comb_test)
print(f"Test Accuracy on Combined Model: {acc_test:.3f}")
print("Classification Report (Test):")
print(classification_report(y_test_comb, y_pred_comb_test))

# Step 5.3: Fairness Mitigation (Optional)
# e.g., Reweighing or Post-Processing
# from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, etc.

# Pseudocode demonstration
# privileged_groups = [{'NEWRACE_Black': 0}]
# unprivileged_groups = [{'NEWRACE_Black': 1}]
# binary_label_dataset_test = BinaryLabelDataset(...)

# ceo = CalibratedEqOddsPostprocessing(privileged_groups=privileged_groups, 
#                                       unprivileged_groups=unprivileged_groups, 
#                                       cost_constraint="fnr", seed=42)
# ceo.fit(binary_label_dataset_train, binary_label_dataset_val)
# pred_ceo = ceo.predict(binary_label_dataset_test)

# Step 5.4: Evaluate Fairness (TPR, FPR, Demographic Parity, etc.)
# Example for a single protected attribute 'NEWRACE_Black' in your structured data
print("\n=== Fairness Metrics on Combined Model ===")
protected_col_idx = ... # Find the index of 'NEWRACE_Black' in your structured_features

def group_accuracy(y_true, y_pred, X_array, idx_protected, group_val):
    mask = (X_array[:, idx_protected] == group_val)
    return accuracy_score(y_true[mask], y_pred[mask])

acc_priv = group_accuracy(y_test_comb, y_pred_comb_test, X_test_comb, protected_col_idx, 0)
acc_unpriv = group_accuracy(y_test_comb, y_pred_comb_test, X_test_comb, protected_col_idx, 1)
print("Accuracy (Unprivileged):", acc_unpriv)
print("Accuracy (Privileged):  ", acc_priv)

# Additional TPR/FPR calculations omitted for brevity
# You can incorporate the confusion matrix approach with 'labels=[0,1]' as in Phase 4.

print("\n=== Phase 5 Completed ===\n")
print("Advanced model (BERT-finetuned embeddings + structured data + optional fairness mitigation) is trained.")
print("Evaluate performance & fairness. If biases remain, apply further strategies (post-processing, adversarial, etc.).")